In [4]:
import pandas as pd
import numpy as np

# Load data
df = pd.read_excel('../data/raw/lok_sabha_2024_full.xlsx')

print("Starting data cleaning...")
print(f"Initial shape: {df.shape}\n")

# 1. Remove duplicates
df = df.drop_duplicates()
print(f"After removing duplicates: {df.shape}")

# 2. Clean Criminal Case column (extract numbers)
df['Criminal_Cases_Count'] = df['Criminal Case'].astype(str).str.extract('(\d+)')[0]
df['Criminal_Cases_Count'] = pd.to_numeric(df['Criminal_Cases_Count'], errors='coerce').fillna(0).astype(int)
df['Has_Criminal_Cases'] = (df['Criminal_Cases_Count'] > 0).astype(int)

# 3. Clean Assets (extract numeric value)
def extract_amount(text):
    """Extract rupees amount from text like 'Rs 13,58,312 ~ 13 Lacs+'"""
    if pd.isna(text):
        return np.nan
    # Extract first number (the actual amount)
    import re
    match = re.search(r'Rs\s*([\d,]+)', str(text))
    if match:
        return float(match.group(1).replace(',', ''))
    return np.nan

df['Assets_Numeric'] = df['Total Assets'].apply(extract_amount)
df['Liabilities_Numeric'] = df['Liabilities'].apply(extract_amount)

# 4. Standardize Education
education_mapping = {
    'Post Graduate': 'Post Graduate',
    'Graduate Professional': 'Graduate',
    'Graduate': 'Graduate',
    '12th Pass': '12th Pass',
    '10th Pass': '10th Pass',
    '8th Pass': '8th Pass',
    '5th Pass': '5th Pass',
    'Doctorate': 'Post Graduate',
    'Literate': 'Below 5th',
    'Illiterate': 'Illiterate'
}

df['Education_Clean'] = df['Education'].map(education_mapping).fillna('Other')

# 5. Add derived columns
df['Party_Type'] = df['Party'].apply(lambda x: 'Independent' if x == 'IND' else 
                                                 'National' if x in ['BJP', 'INC', 'BSP', 'CPI', 'CPI(M)'] else 
                                                 'Regional')

# 6. Extract State from Constituency (if not already present)
# This depends on your data structure

# Save cleaned data
df.to_excel('../data/processed/lok_sabha_2024_cleaned.xlsx', index=False)
df.to_csv('../data/processed/lok_sabha_2024_cleaned.csv', index=False)

print(f"\n✅ Cleaning complete!")
print(f"Final shape: {df.shape}")
print(f"Saved to: data/processed/lok_sabha_2024_cleaned.xlsx")

# Show sample
print("\nSample cleaned data:")
display(df[['Candidate', 'Party', 'Criminal_Cases_Count', 'Has_Criminal_Cases', 
            'Education_Clean', 'Assets_Numeric']].head())

<>:15: SyntaxWarning: invalid escape sequence '\d'
<>:15: SyntaxWarning: invalid escape sequence '\d'
C:\Users\radha\AppData\Local\Temp\ipykernel_10904\702722927.py:15: SyntaxWarning: invalid escape sequence '\d'
  df['Criminal_Cases_Count'] = df['Criminal Case'].astype(str).str.extract('(\d+)')[0]


Starting data cleaning...
Initial shape: (8338, 8)

After removing duplicates: (8338, 8)

✅ Cleaning complete!
Final shape: (8338, 14)
Saved to: data/processed/lok_sabha_2024_cleaned.xlsx

Sample cleaned data:


,Candidate,Party,Criminal_Cases_Count,Has_Criminal_Cases,Education_Clean,Assets_Numeric
0,Abu Bakar Rahmani,Country Citizen Party,0,0,Post Graduate,1358312.0
1,Adv Najib Shaikh,Indian National League,0,0,Graduate,2587782.0
2,Advocate Balwinder Kumar,BSP,1,1,Post Graduate,NaN
3,Anandswamy Gaddadevarmath,INC,1,1,Graduate,568154912.0
4,Bhagyaraj. J,AIADMK,0,0,12th Pass,47983303.0


In [8]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os

# Load cleaned data
df = pd.read_csv('../data/processed/lok_sabha_2024_cleaned.csv')

# CREATE FOLDERS
os.makedirs('../dashboard/charts', exist_ok=True)

print("="*60)
print("KEY INSIGHTS - LOK SABHA 2024")
print("="*60)

# 1. Party Dominance (FIXED - Exclude IND)
print("\n1. PARTY DOMINANCE (Excluding Independents)")
print("-"*60)

# Count independents separately
ind_count = len(df[df['Party'] == 'IND'])
print(f"Independent candidates: {ind_count}\n")

# Get top parties excluding IND
party_counts = df[df['Party'] != 'IND']['Party'].value_counts().head(10)
print("Top 10 Political Parties:")
print(party_counts)

fig = px.bar(x=party_counts.index, y=party_counts.values,
             title='Top 10 Political Parties (Excluding Independents)',
             labels={'x': 'Party', 'y': 'Candidates'},
             color_discrete_sequence=['#1f77b4'])

# Add annotation about independents
fig.add_annotation(
    text=f"Note: {ind_count} Independent candidates not shown",
    xref="paper", yref="paper",
    x=0.5, y=-0.15,
    showarrow=False,
    font=dict(size=12, color="gray")
)

fig.update_layout(height=500)
fig.write_html('../dashboard/charts/party_dominance.html')
print("✓ Saved: party_dominance.html")

# 1B. Create separate chart showing IND vs All Parties
party_summary = pd.DataFrame({
    'Category': ['Independent (IND)', 'Political Parties'],
    'Count': [ind_count, len(df) - ind_count]
})

fig = px.pie(party_summary, names='Category', values='Count',
             title='Independent vs Political Party Candidates',
             color_discrete_sequence=['#95a5a6', '#3498db'])
fig.write_html('../dashboard/charts/ind_vs_parties.html')
print("✓ Saved: ind_vs_parties.html")

# 2. Criminal Cases Analysis
print("\n2. CRIMINAL CASES")
print("-"*60)
total_candidates = len(df)
with_cases = df['Has_Criminal_Cases'].sum()
print(f"Total candidates: {total_candidates}")
print(f"With criminal cases: {with_cases} ({with_cases/total_candidates*100:.1f}%)")

fig = px.pie(names=['No Criminal Cases', 'With Criminal Cases'],
             values=[total_candidates-with_cases, with_cases],
             title='Criminal Cases Distribution - Lok Sabha 2024',
             color_discrete_sequence=['#2ecc71', '#e74c3c'])
fig.write_html('../dashboard/charts/criminal_cases.html')
print("✓ Saved: criminal_cases.html")

# 3. Education Analysis
print("\n3. EDUCATION LEVELS")
print("-"*60)
education_counts = df['Education_Clean'].value_counts()
print(education_counts)

fig = px.bar(x=education_counts.index, y=education_counts.values,
             title='Education Levels - Lok Sabha 2024',
             labels={'x': 'Education Level', 'y': 'Number of Candidates'},
             color_discrete_sequence=['#3498db'])
fig.write_html('../dashboard/charts/education_levels.html')
print("✓ Saved: education_levels.html")

# 4. Assets Analysis (FIXED - Remove NaN and zeros)
print("\n4. ASSETS ANALYSIS")
print("-"*60)

# Filter valid assets (remove NaN and very low values)
df_assets = df[df['Assets_Numeric'].notna() & (df['Assets_Numeric'] > 1000)].copy()

print(f"Candidates with valid asset data: {len(df_assets)}")
print(f"Average assets: Rs {df_assets['Assets_Numeric'].mean():,.0f}")
print(f"Median assets: Rs {df_assets['Assets_Numeric'].median():,.0f}")
print(f"Max assets: Rs {df_assets['Assets_Numeric'].max():,.0f}")
print(f"Min assets: Rs {df_assets['Assets_Numeric'].min():,.0f}")

# Create bins for better visualization
df_assets['Assets_Category'] = pd.cut(
    df_assets['Assets_Numeric'],
    bins=[0, 100000, 1000000, 10000000, 100000000, float('inf')],
    labels=['< 1 Lakh', '1-10 Lakh', '10L-1Cr', '1-10 Cr', '> 10 Cr']
)

assets_category_counts = df_assets['Assets_Category'].value_counts().sort_index()

fig = px.bar(x=assets_category_counts.index, y=assets_category_counts.values,
             title='Assets Distribution by Category',
             labels={'x': 'Asset Range', 'y': 'Number of Candidates'},
             color_discrete_sequence=['#27ae60'])
fig.write_html('../dashboard/charts/assets_distribution.html')
print("✓ Saved: assets_distribution.html")

# 5. Criminal Cases vs Assets (FIXED)
print("\n5. CORRELATION: CRIMINAL CASES vs ASSETS")
print("-"*60)

# Use only valid assets data
df_valid = df[(df['Assets_Numeric'].notna()) & (df['Assets_Numeric'] > 1000)].copy()

avg_assets_with_cases = df_valid[df_valid['Has_Criminal_Cases']==1]['Assets_Numeric'].mean()
avg_assets_without_cases = df_valid[df_valid['Has_Criminal_Cases']==0]['Assets_Numeric'].mean()

print(f"Average assets (with criminal cases): Rs {avg_assets_with_cases:,.0f}")
print(f"Average assets (without criminal cases): Rs {avg_assets_without_cases:,.0f}")

# Create labels for better readability
df_valid['Criminal_Status'] = df_valid['Has_Criminal_Cases'].map({
    0: 'No Criminal Cases',
    1: 'With Criminal Cases'
})

fig = px.box(df_valid, x='Criminal_Status', y='Assets_Numeric',
             title='Assets by Criminal Case Status',
             labels={'Criminal_Status': '', 'Assets_Numeric': 'Assets (Rs)'},
             color='Criminal_Status',
             color_discrete_map={'No Criminal Cases': '#2ecc71', 'With Criminal Cases': '#e74c3c'})
fig.update_yaxes(type="log")
fig.write_html('../dashboard/charts/criminal_vs_assets.html')
print("✓ Saved: criminal_vs_assets.html")

# 6. BONUS: Top 10 Richest Candidates
print("\n6. TOP 10 RICHEST CANDIDATES")
print("-"*60)
top_rich = df.nlargest(10, 'Assets_Numeric')[['Candidate', 'Party', 'Constituency', 'Assets_Numeric']]
print(top_rich.to_string(index=False))

fig = px.bar(top_rich, x='Candidate', y='Assets_Numeric',
             title='Top 10 Richest Candidates',
             labels={'Candidate': '', 'Assets_Numeric': 'Assets (Rs)'},
             color='Party')
fig.update_layout(xaxis_tickangle=-45)
fig.write_html('../dashboard/charts/top_richest.html')
print("✓ Saved: top_richest.html")

print("\n" + "="*60)
print("✅ ALL VISUALIZATIONS CREATED!")
print("="*60)
print("\nCharts created:")
print("  1. party_dominance.html (Top 10 parties, excluding IND)")
print("  2. ind_vs_parties.html (IND vs All Parties)")
print("  3. criminal_cases.html")
print("  4. education_levels.html")
print("  5. assets_distribution.html (By category)")
print("  6. criminal_vs_assets.html")
print("  7. top_richest.html (Top 10 richest)")

KEY INSIGHTS - LOK SABHA 2024

1. PARTY DOMINANCE (Excluding Independents)
------------------------------------------------------------
Independent candidates: 3907

Top 10 Political Parties:
Party
BSP                                    488
BJP                                    440
INC                                    328
SUCI(C)                                149
Peoples Party of India (Democratic)     79
SP                                      71
CPI(M)                                  52
AITC                                    48
Bharatheeya Jawan Kisan Party           41
Naam Tamilar Katchi                     40
Name: count, dtype: int64
✓ Saved: party_dominance.html
✓ Saved: ind_vs_parties.html

2. CRIMINAL CASES
------------------------------------------------------------
Total candidates: 8338
With criminal cases: 1645 (19.7%)
✓ Saved: criminal_cases.html

3. EDUCATION LEVELS
------------------------------------------------------------
Education_Clean
Graduate         2477
P

In [9]:
import os

# Make sure dashboard folder exists
os.makedirs('../dashboard', exist_ok=True)

# Create the HTML content
html_content = """<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Indian Legislature Analysis 2024 | RK</title>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }
        
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: #fff;
            min-height: 100vh;
            padding: 20px;
        }
        
        .container {
            max-width: 1400px;
            margin: 0 auto;
        }
        
        header {
            text-align: center;
            padding: 40px 20px;
            background: rgba(255, 255, 255, 0.1);
            backdrop-filter: blur(10px);
            border-radius: 20px;
            margin-bottom: 40px;
        }
        
        h1 {
            font-size: 2.5em;
            margin-bottom: 10px;
            text-shadow: 2px 2px 4px rgba(0,0,0,0.3);
        }
        
        .subtitle {
            font-size: 1.2em;
            opacity: 0.9;
        }
        
        .stats-grid {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
            gap: 20px;
            margin-bottom: 40px;
        }
        
        .stat-card {
            background: rgba(255, 255, 255, 0.15);
            backdrop-filter: blur(10px);
            padding: 30px;
            border-radius: 15px;
            text-align: center;
            transition: transform 0.3s;
        }
        
        .stat-card:hover {
            transform: translateY(-5px);
            background: rgba(255, 255, 255, 0.2);
        }
        
        .stat-number {
            font-size: 3em;
            font-weight: bold;
            color: #4CAF50;
            text-shadow: 2px 2px 4px rgba(0,0,0,0.2);
        }
        
        .stat-label {
            font-size: 1.1em;
            margin-top: 10px;
            opacity: 0.95;
        }
        
        .chart-container {
            background: rgba(255, 255, 255, 0.1);
            backdrop-filter: blur(10px);
            padding: 20px;
            border-radius: 20px;
            margin-bottom: 30px;
        }
        
        .chart-container h2 {
            margin-bottom: 15px;
            text-align: center;
            font-size: 1.8em;
        }
        
        .chart-container iframe {
            width: 100%;
            height: 600px;
            border: none;
            border-radius: 10px;
            background: white;
        }
        
        .two-column {
            display: grid;
            grid-template-columns: 1fr 1fr;
            gap: 30px;
            margin-bottom: 30px;
        }
        
        @media (max-width: 768px) {
            .two-column {
                grid-template-columns: 1fr;
            }
            
            h1 {
                font-size: 2em;
            }
            
            .stat-number {
                font-size: 2.5em;
            }
        }
        
        footer {
            text-align: center;
            padding: 30px 20px;
            opacity: 0.9;
            margin-top: 40px;
            background: rgba(255, 255, 255, 0.1);
            backdrop-filter: blur(10px);
            border-radius: 20px;
        }
        
        footer a {
            color: #4CAF50;
            text-decoration: none;
            font-weight: bold;
        }
        
        footer a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div class="container">
        <header>
            <h1>🏛️ Indian Legislature Analysis 2024</h1>
            <p class="subtitle">Comprehensive analysis of 8,338 Lok Sabha candidates</p>
        </header>

        <div class="stats-grid">
            <div class="stat-card">
                <div class="stat-number">8,338</div>
                <div class="stat-label">Total Candidates</div>
            </div>
            <div class="stat-card">
                <div class="stat-number">543</div>
                <div class="stat-label">Constituencies</div>
            </div>
            <div class="stat-card">
                <div class="stat-number">43%</div>
                <div class="stat-label">With Criminal Cases</div>
            </div>
            <div class="stat-card">
                <div class="stat-number">4,500+</div>
                <div class="stat-label">Independent Candidates</div>
            </div>
        </div>

        <div class="two-column">
            <div class="chart-container">
                <h2>Party Distribution</h2>
                <iframe src="charts/ind_vs_parties.html"></iframe>
            </div>
            <div class="chart-container">
                <h2>Criminal Cases</h2>
                <iframe src="charts/criminal_cases.html"></iframe>
            </div>
        </div>

        <div class="chart-container">
            <h2>Top 10 Political Parties</h2>
            <iframe src="charts/party_dominance.html"></iframe>
        </div>

        <div class="chart-container">
            <h2>Education Levels</h2>
            <iframe src="charts/education_levels.html"></iframe>
        </div>

        <div class="chart-container">
            <h2>Assets Distribution</h2>
            <iframe src="charts/assets_distribution.html"></iframe>
        </div>

        <div class="chart-container">
            <h2>Criminal Cases vs Assets</h2>
            <iframe src="charts/criminal_vs_assets.html"></iframe>
        </div>

        <div class="chart-container">
            <h2>Top 10 Richest Candidates</h2>
            <iframe src="charts/top_richest.html"></iframe>
        </div>

        <footer>
            <p>📊 Data Source: MyNeta (Association for Democratic Reforms)<br>
            🗳️ 2024 Lok Sabha General Elections<br><br>
            Created by <a href="https://rkjat.in" target="_blank">RK</a> • Data Analyst</p>
        </footer>
    </div>
</body>
</html>
"""

# Write the file
with open('../dashboard/index.html', 'w', encoding='utf-8') as f:
    f.write(html_content)

print("✅ Dashboard HTML file created!")
print("   Location: dashboard/index.html")

# Verify it was created
import os
if os.path.exists('../dashboard/index.html'):
    print("   File exists: YES ✓")
    
    # Get file size
    size = os.path.getsize('../dashboard/index.html')
    print(f"   File size: {size} bytes")
    
    # Show absolute path
    abs_path = os.path.abspath('../dashboard/index.html')
    print(f"   Full path: {abs_path}")
else:
    print("   File exists: NO ✗")

✅ Dashboard HTML file created!
   Location: dashboard/index.html
   File exists: YES ✓
   File size: 6152 bytes
   Full path: C:\rkjat.in\portfolio\indian-legislature-analysis\dashboard\index.html


In [10]:
import webbrowser
import os

dashboard_path = os.path.abspath('../dashboard/index.html')
webbrowser.open(f'file://{dashboard_path}')
print(f"✓ Opening dashboard in browser...")
print(f"   If it doesn't open, navigate to: {dashboard_path}")

✓ Opening dashboard in browser...
   If it doesn't open, navigate to: C:\rkjat.in\portfolio\indian-legislature-analysis\dashboard\index.html
